In [1]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np, pandas as pd
from fbprophet import Prophet
import math
from sklearn.metrics import mean_squared_error, mean_absolute_error
from matplotlib import pyplot
from pmdarima.arima.utils import ndiffs
from statsmodels.tsa.stattools import adfuller

import warnings
warnings.filterwarnings("ignore")

In [7]:
#DATA PREPARATION FOR UNIVARIATE PROPHET MODEL

#GATHER Delhi DATA FROM ALL AVAILABLE YEARS
Delhi_Data = pd.read_csv('Delhi_data.csv')
Delhi_Data.rename(columns = {Delhi_Data.columns[0]:'ds'}, inplace = True)

#KEEP ONLY THE POLLUTANTS COLUMNS
cols=['pm25', 'pm10', 'no2', 'o3', 'so2', 'ds']

#MAKE VALUES NUMERIC
Delhi_Data_uni = Delhi_Data.loc[:, Delhi_Data.columns.intersection(cols)]
Delhi_Data_uni[['pm25', 'pm10', 'no2', 'o3', 'so2']] = Delhi_Data_uni[['pm25', 'pm10', 'no2', 'o3', 'so2']].apply(pd.to_numeric, errors='coerce')

#FILL NAN VALUES
Delhi_Data_uni = Delhi_Data_uni.iloc[1: , :]

#SOME VALUES ARE EMPTY, BUT NOT RECOGNIZED AS NAN. REPLACE EMPTY VALUES WITH NAN TO FILL THEM LATER
Delhi_Data_uni = Delhi_Data_uni.replace(r'^\s*$', np.nan, regex=True)
Delhi_Data_uni = Delhi_Data_uni.fillna(method='ffill')
Delhi_Data_uni = Delhi_Data_uni.fillna(method='bfill')

#REMOVE DUPLICATE VALUES
Delhi_Data_uni = Delhi_Data_uni[~Delhi_Data_uni.index.duplicated(keep='first')]

#Fb prophet model predicts univariate time series. It requires that the Date index column is named 'ds'
#and the values column is named 'y'
Delhi_Data_uni['ds'] = pd.DatetimeIndex(Delhi_Data_uni['ds'])

In [3]:
Delhi_Data_uni

,ds,no2,o3,pm10,pm25,so2
1,2014-12-30,0.516287,0.108040,0.319430,0.616088,0.202970
2,2014-12-31,0.384365,0.183417,0.123093,0.314442,0.227723
3,2015-01-01,1.000000,0.118090,0.217701,0.643510,0.405941
4,2015-01-02,0.469055,0.138191,0.066124,0.259598,0.123762
5,2015-01-03,0.369707,0.118090,0.083418,0.272395,0.193069
...,...,...,...,...,...,...
1895,2022-03-15,0.179153,0.175879,0.128179,0.266910,0.252475
1896,2022-03-16,0.133550,0.195980,0.115972,0.246801,0.232673
1897,2022-03-17,0.177524,0.203518,0.130214,0.272395,0.306931
1898,2022-03-18,0.076547,0.278894,0.124110,0.283364,0.217822


In [4]:
#Check data for stationarity to apply the ARIMA model.

#The null hypothesis is that the time series is non-stationary.
#Using the Augmented Dickey Fuller test (adfuller()) we can check for stationarity.
#If p-value < significance level(0.05), the we reject the null hypothesis.
#Otherwise, the time series is non-stationary and needs differencing.

print("Check if Delhi_Data is stationary :")
for col in ['no2', 'o3', 'so2', 'pm10', 'pm25']:
    result = adfuller(Delhi_Data[col])
    print('p-value for ' + col + ' : %f' % result[1])

Check if Delhi_Data is stationary :
p-value for no2 : 0.000491
p-value for o3 : 0.003891
p-value for so2 : 0.000114
p-value for pm10 : 0.000009
p-value for pm25 : 0.000003


In [3]:
def fbprophet_predict(time_series, col):
    
    #CREATE MODEL
    model = Prophet()
    
    #SPLIT TO TRAIN AND TEST PORTIONS
    train_size = 0.8
    train = time_series[:int(train_size*(len(time_series)))]
    test = time_series[int(train_size*(len(time_series))):]
    
    #FIT TRAIN DATA TO MODEL
    model.fit(train)
    
    #PREDICT THE TEST PORTION. THE PARAMETER IN THE PREDICT FUNCTION IS THE TIME INTERVAL TO PREDICT
    forecast = model.predict(test[['ds']])
    
    #FORECAST PREVIEW. 'ds' IS THE DAILY INDEX, 'yhat' IS THE FORECAST, 'yhat_lower' and 'yhat_upper' ARE THE
    #LOWER AND UPPER BOUND OF THE FORECASTED VALUE RESPECTIVELY.
    print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].head())
    print(test.reset_index(drop=True).head())
    
    #COMPUTE RMSE OF THE FORECASTED AND ACTUAL VALUE
    rmse = math.sqrt(mean_squared_error(test[['y']], forecast[['yhat']]))
    mse = mean_squared_error(test[['y']], forecast[['yhat']])
    mae = mean_absolute_error(test[['y']], forecast[['yhat']])
    
    print("FB Prophet RMSE for Delhi[" + col + "]: " + str(rmse) + "\n")
    print("FB Prophet MSE for Delhi[" + col + "]: " + str(mse) + "\n")
    print("FB Prophet MAE for Delhi[" + col + "]: " + str(mae) + "\n")

    return test, forecast, rmse

In [6]:
#FB PROPHET REQUIRES THE DATAFRAME TO HAVE TWO COLUMNS. 'ds' AND 'y', 'ds' IS THE TIME INDEX AND 'y' IS THE
#VALUE OF THE POLLUTANT

#RENAME COLUMNS TO FIT TO THE PROPHET MODEL
Delhi_o3 = Delhi_Data_uni[['ds', 'o3']]
Delhi_o3.rename(columns = {'o3':'y'}, inplace = True)

Delhi_no2 = Delhi_Data_uni[['ds', 'no2']]
Delhi_no2.rename(columns = {'no2':'y'}, inplace = True)

Delhi_so2 = Delhi_Data_uni[['ds', 'so2']]
Delhi_so2.rename(columns = {'so2':'y'}, inplace = True)

Delhi_pm10 = Delhi_Data_uni[['ds', 'pm10']]
Delhi_pm10.rename(columns = {'pm10':'y'}, inplace = True)

Delhi_pm25 = Delhi_Data_uni[['ds', 'pm25']]
Delhi_pm25.rename(columns = {'pm25':'y'}, inplace = True)

datasets = [Delhi_o3, Delhi_no2, Delhi_so2, Delhi_pm10, Delhi_pm25]

In [7]:
pollutants = ['o3', 'no2', 'so2', 'pm10', 'pm25']

i=0

#EVALUATE MODEL FOR EVERY POLLUTANT
for data in datasets:
    test, forecast, error = fbprophet_predict(data, pollutants[i])
    i = i+1

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2021-02-11  0.181474    0.054950    0.308126
1 2021-02-12  0.186484    0.071055    0.314342
2 2021-02-13  0.189076    0.057794    0.312866
3 2021-02-14  0.215687    0.087927    0.333203
4 2021-02-15  0.212708    0.088497    0.334218
          ds         y
0 2021-02-11  0.108040
1 2021-02-12  0.097990
2 2021-02-13  0.120603
3 2021-02-14  0.133166
4 2021-02-15  0.135678
FB Prophet RMSE for Delhi[o3]: 0.07609657941715994

FB Prophet MSE for Delhi[o3]: 0.005790689398992131

FB Prophet MAE for Delhi[o3]: 0.061155730012919615



INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2021-02-11  0.246858    0.131118    0.355728
1 2021-02-12  0.238284    0.131544    0.355498
2 2021-02-13  0.235104    0.120620    0.342746
3 2021-02-14  0.208221    0.093016    0.326006
4 2021-02-15  0.222618    0.102480    0.330668
          ds         y
0 2021-02-11  0.286645
1 2021-02-12  0.273616
2 2021-02-13  0.255700
3 2021-02-14  0.236156
4 2021-02-15  0.278502
FB Prophet RMSE for Delhi[no2]: 0.0797064344131809

FB Prophet MSE for Delhi[no2]: 0.0063531156868627085

FB Prophet MAE for Delhi[no2]: 0.06507966150033927



INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2021-02-11  0.247374    0.140463    0.361261
1 2021-02-12  0.243169    0.140418    0.349522
2 2021-02-13  0.244774    0.131204    0.351161
3 2021-02-14  0.233335    0.125592    0.348116
4 2021-02-15  0.233147    0.128667    0.345301
          ds         y
0 2021-02-11  0.272277
1 2021-02-12  0.193069
2 2021-02-13  0.133663
3 2021-02-14  0.158416
4 2021-02-15  0.193069
FB Prophet RMSE for Delhi[so2]: 0.07033535933945917

FB Prophet MSE for Delhi[so2]: 0.004947062773410846

FB Prophet MAE for Delhi[so2]: 0.05624727962303807



INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2021-02-11  0.119434    0.028439    0.200088
1 2021-02-12  0.114961    0.033595    0.204057
2 2021-02-13  0.107268    0.017956    0.192938
3 2021-02-14  0.096771    0.013327    0.184217
4 2021-02-15  0.096757    0.014850    0.184475
          ds         y
0 2021-02-11  0.158698
1 2021-02-12  0.189217
2 2021-02-13  0.158698
3 2021-02-14  0.167854
4 2021-02-15  0.160732
FB Prophet RMSE for Delhi[pm10]: 0.05550276669227234

FB Prophet MSE for Delhi[pm10]: 0.0030805571104968155

FB Prophet MAE for Delhi[pm10]: 0.04319320802621093

          ds      yhat  yhat_lower  yhat_upper
0 2021-02-11  0.290086    0.196118    0.385114
1 2021-02-12  0.285182    0.189191    0.378793
2 2021-02-13  0.276877    0.187617    0.366269
3 2021-02-14  0.269449    0.174410    0.358750
4 2021-02-15  0.265467    0.184697    0.359570
          ds         y
0 2021-02-11  0.352834
1 2021-02-12  0.436929
2 2021-02-13  0.371115
3 2021-02-14  0.380256
4 2021-02-15  0.34734

### Repeat process but this time train model with the same amount of data as Athens, to prove that the model returns better results due to bigger amount of data.

In [8]:
import pandas as pd
Athens_Data = pd.read_csv('Athens_Data.csv', index_col=0)

In [9]:
len(Athens_Data)

839

In [10]:
data_to_drop = len(Delhi_Data) - len(Athens_Data)

In [11]:
Delhi_less_data = Delhi_Data.iloc[data_to_drop: , :]
len(Delhi_less_data)

839

In [12]:
Delhi_less_data

,ds,no2,o3,pm10,pm25,so2,temp,dew,humidity,windspeed,winddir
1061,2019-11-10,0.258958,0.123116,0.170905,0.402194,0.292079,0.457317,0.372180,0.458583,0.036290,0.589863
1062,2019-11-11,0.315961,0.108040,0.185148,0.436929,0.297030,0.432927,0.353383,0.464586,0.029234,0.460820
1063,2019-11-12,0.350163,0.110553,0.427263,0.703839,0.282178,0.408537,0.360902,0.506603,0.038306,0.407980
1064,2019-11-13,0.395765,0.085427,0.436419,0.731261,0.287129,0.432927,0.436090,0.561825,0.020161,0.245866
1065,2019-11-14,0.477199,0.143216,0.447609,0.714808,0.311881,0.435976,0.511278,0.644658,0.020161,0.159597
...,...,...,...,...,...,...,...,...,...,...,...
1895,2022-03-15,0.179153,0.175879,0.128179,0.266910,0.252475,0.618902,0.699248,0.642257,0.065524,0.572969
1896,2022-03-16,0.133550,0.195980,0.115972,0.246801,0.232673,0.640244,0.684211,0.589436,0.095766,0.861970
1897,2022-03-17,0.177524,0.203518,0.130214,0.272395,0.306931,0.643293,0.706767,0.602641,0.020161,0.511503
1898,2022-03-18,0.076547,0.278894,0.124110,0.283364,0.217822,0.658537,0.774436,0.656663,0.045867,0.338246


In [13]:
Delhi_o3 = Delhi_less_data[['ds', 'o3']]
Delhi_o3.rename(columns = {'o3':'y'}, inplace = True)

Delhi_no2 = Delhi_less_data[['ds', 'no2']]
Delhi_no2.rename(columns = {'no2':'y'}, inplace = True)

Delhi_so2 = Delhi_less_data[['ds', 'so2']]
Delhi_so2.rename(columns = {'so2':'y'}, inplace = True)

Delhi_pm10 = Delhi_less_data[['ds', 'pm10']]
Delhi_pm10.rename(columns = {'pm10':'y'}, inplace = True)

Delhi_pm25 = Delhi_less_data[['ds', 'pm25']]
Delhi_pm25.rename(columns = {'pm25':'y'}, inplace = True)

datasets = [Delhi_o3, Delhi_no2, Delhi_so2, Delhi_pm10, Delhi_pm25]

i=0

#EVALUATE MODEL FOR EVERY POLLUTANT
for data in datasets:
    test, forecast, error = fbprophet_predict(data, pollutants[i])
    i = i+1

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2021-09-11  0.240844    0.118203    0.351333
1 2021-09-12  0.245428    0.134719    0.358234
2 2021-09-13  0.236795    0.127795    0.352782
3 2021-09-14  0.230171    0.113732    0.349324
4 2021-09-15  0.233597    0.119139    0.345802
           ds         y
0  2021-09-11  0.185930
1  2021-09-12  0.165829
2  2021-09-13  0.178392
3  2021-09-14  0.160804
4  2021-09-15  0.180905
FB Prophet RMSE for Delhi[o3]: 0.09731588051161101

FB Prophet MSE for Delhi[o3]: 0.009470380599750153

FB Prophet MAE for Delhi[o3]: 0.08556797351431669



INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2021-09-11  0.006220   -0.092738    0.108562
1 2021-09-12 -0.001895   -0.101127    0.098890
2 2021-09-13  0.009821   -0.084543    0.104246
3 2021-09-14  0.015304   -0.086389    0.110598
4 2021-09-15  0.013445   -0.084999    0.108261
           ds         y
0  2021-09-11  0.048860
1  2021-09-12  0.052117
2  2021-09-13  0.071661
3  2021-09-14  0.048860
4  2021-09-15  0.040717
FB Prophet RMSE for Delhi[no2]: 0.31774477431148296

FB Prophet MSE for Delhi[no2]: 0.10096174160225523

FB Prophet MAE for Delhi[no2]: 0.2954301214518543



INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2021-09-11  0.020200   -0.066618    0.103275
1 2021-09-12  0.023460   -0.060776    0.107245
2 2021-09-13  0.027466   -0.051028    0.107775
3 2021-09-14  0.031059   -0.049323    0.116011
4 2021-09-15  0.026356   -0.051460    0.102574
           ds         y
0  2021-09-11  0.049505
1  2021-09-12  0.064356
2  2021-09-13  0.074257
3  2021-09-14  0.089109
4  2021-09-15  0.074257
FB Prophet RMSE for Delhi[so2]: 0.29516459665071715

FB Prophet MSE for Delhi[so2]: 0.08712213911598055

FB Prophet MAE for Delhi[so2]: 0.27024153279055585



INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2021-09-11  0.030558   -0.046562    0.115623
1 2021-09-12  0.029752   -0.055708    0.109218
2 2021-09-13  0.032098   -0.048401    0.115021
3 2021-09-14  0.042238   -0.037795    0.132705
4 2021-09-15  0.038656   -0.043886    0.122347
           ds         y
0  2021-09-11  0.012208
1  2021-09-12  0.029502
2  2021-09-13  0.044761
3  2021-09-14  0.040692
4  2021-09-15  0.036623
FB Prophet RMSE for Delhi[pm10]: 0.18039696467683353

FB Prophet MSE for Delhi[pm10]: 0.032543064864614724

FB Prophet MAE for Delhi[pm10]: 0.15730390973725303

          ds      yhat  yhat_lower  yhat_upper
0 2021-09-11  0.067710   -0.035589    0.193351
1 2021-09-12  0.069918   -0.050769    0.183920
2 2021-09-13  0.069938   -0.043956    0.184848
3 2021-09-14  0.073543   -0.045411    0.185038
4 2021-09-15  0.069440   -0.036615    0.184207
           ds         y
0  2021-09-11  0.056673
1  2021-09-12  0.087751
2  2021-09-13  0.131627
3  2021-09-14  0.076782
4  2021-09-

## Seasonal split

The seasonal decomposition of the Delhi data we can see that there is a seasonality in the monthly resampled data. Specifically, for each pollutant the seasonal length of the data is:
- NO2: 12 months or 365 days
- SO2: 12 months
- O3: 12 months
- PM25: 12 months
- PM10: 12 months

In [14]:
def fbprophet_predict_seasonal(time_series, col):
    
    #CREATE MODEL
    model = Prophet()
    time_series.index = time_series['ds']
    #SPLIT TO TRAIN AND TEST PORTIONS
    train, test = time_series['2019-12-01' : '2021-12-31'], time_series['2022-01-01' : '2022-03-18']
    
    #FIT TRAIN DATA TO MODEL
    model.fit(train)
    
    #PREDICT THE TEST PORTION. THE PARAMETER IN THE PREDICT FUNCTION IS THE TIME INTERVAL TO PREDICT
    forecast = model.predict(test[['ds']])
    
    #FORECAST PREVIEW. 'ds' IS THE DAILY INDEX, 'yhat' IS THE FORECAST, 'yhat_lower' and 'yhat_upper' ARE THE
    #LOWER AND UPPER BOUND OF THE FORECASTED VALUE RESPECTIVELY.
    print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].head())
    print(test.reset_index(drop=True).head())
    
    #COMPUTE RMSE OF THE FORECASTED AND ACTUAL VALUE
    rmse = math.sqrt(mean_squared_error(test[['y']], forecast[['yhat']]))
    mse = mean_squared_error(test[['y']], forecast[['yhat']])
    mae = mean_absolute_error(test[['y']], forecast[['yhat']])
    
    print("FB Prophet RMSE for Delhi[" + col + "]: " + str(rmse) + "\n")
    print("FB Prophet MSE for Delhi[" + col + "]: " + str(mse) + "\n")
    print("FB Prophet MAE for Delhi[" + col + "]: " + str(mae) + "\n")

    return test, forecast, rmse

In [15]:
pollutants = ['o3', 'no2', 'so2', 'pm10', 'pm25']

i=0

#EVALUATE MODEL FOR EVERY POLLUTANT
for data in datasets:
    test, forecast, error = fbprophet_predict_seasonal(data, pollutants[i])
    
    i = i+1

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2022-01-01  0.097106    0.012253    0.182511
1 2022-01-02  0.099065    0.008684    0.175259
2 2022-01-03  0.093584    0.008217    0.175603
3 2022-01-04  0.088262   -0.000419    0.171118
4 2022-01-05  0.090807    0.002040    0.178777
           ds         y
0  2022-01-01  0.057789
1  2022-01-02  0.040201
2  2022-01-03  0.060302
3  2022-01-04  0.070352
4  2022-01-05  0.097990
FB Prophet RMSE for Delhi[o3]: 0.052223362939417615

FB Prophet MSE for Delhi[o3]: 0.0027272796367021376

FB Prophet MAE for Delhi[o3]: 0.036696033778325726



INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2022-01-01  0.231941    0.154076    0.308743
1 2022-01-02  0.221126    0.148387    0.294268
2 2022-01-03  0.228872    0.155823    0.302946
3 2022-01-04  0.230004    0.155386    0.303865
4 2022-01-05  0.228110    0.150884    0.302861
           ds         y
0  2022-01-01  0.312704
1  2022-01-02  0.263844
2  2022-01-03  0.307818
3  2022-01-04  0.324104
4  2022-01-05  0.249186
FB Prophet RMSE for Delhi[no2]: 0.07883749030757338

FB Prophet MSE for Delhi[no2]: 0.006215349877996726

FB Prophet MAE for Delhi[no2]: 0.06769605407172036



INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2022-01-01  0.110194    0.047443    0.168155
1 2022-01-02  0.113974    0.058561    0.176799
2 2022-01-03  0.118530    0.057180    0.176399
3 2022-01-04  0.120939    0.059793    0.180095
4 2022-01-05  0.115485    0.052425    0.176551
           ds         y
0  2022-01-01  0.128713
1  2022-01-02  0.079208
2  2022-01-03  0.133663
3  2022-01-04  0.168317
4  2022-01-05  0.094059
FB Prophet RMSE for Delhi[so2]: 0.042028105528109615

FB Prophet MSE for Delhi[so2]: 0.0017663616542819179

FB Prophet MAE for Delhi[so2]: 0.03353219486343094



INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2022-01-01  0.176257    0.104806    0.246059
1 2022-01-02  0.170213    0.102832    0.237080
2 2022-01-03  0.165563    0.092051    0.237863
3 2022-01-04  0.169238    0.100473    0.239337
4 2022-01-05  0.162814    0.095601    0.229646
           ds         y
0  2022-01-01  0.167854
1  2022-01-02  0.288911
2  2022-01-03  0.184130
3  2022-01-04  0.182096
4  2022-01-05  0.196338
FB Prophet RMSE for Delhi[pm10]: 0.041436850021768966

FB Prophet MSE for Delhi[pm10]: 0.0017170125397265748

FB Prophet MAE for Delhi[pm10]: 0.03176308542609331

          ds      yhat  yhat_lower  yhat_upper
0 2022-01-01  0.401779    0.297486    0.499180
1 2022-01-02  0.398092    0.292844    0.502238
2 2022-01-03  0.389934    0.285950    0.495639
3 2022-01-04  0.386744    0.282518    0.484606
4 2022-01-05  0.378510    0.278460    0.483314
           ds         y
0  2022-01-01  0.429616
1  2022-01-02  0.583181
2  2022-01-03  0.449726
3  2022-01-04  0.431444
4  2022-0

## Weekly predictions

In [4]:
Delhi_Data_uni.index = Delhi_Data_uni['ds']
Delhi_Data_uni = Delhi_Data_uni.drop(['ds'],axis=1)

Delhi_Data_uni.index = pd.to_datetime(Delhi_Data_uni.index)
Delhi_weekly = Delhi_Data_uni.resample("1w").mean().bfill()

Delhi_weekly['ds'] = Delhi_weekly.index

In [5]:
Delhi_o3 = Delhi_weekly[['ds', 'o3']]
Delhi_o3.rename(columns = {'o3':'y'}, inplace = True)

Delhi_no2 = Delhi_weekly[['ds', 'no2']]
Delhi_no2.rename(columns = {'no2':'y'}, inplace = True)

Delhi_so2 = Delhi_weekly[['ds', 'so2']]
Delhi_so2.rename(columns = {'so2':'y'}, inplace = True)

Delhi_pm10 = Delhi_weekly[['ds', 'pm10']]
Delhi_pm10.rename(columns = {'pm10':'y'}, inplace = True)

Delhi_pm25 = Delhi_weekly[['ds', 'pm25']]
Delhi_pm25.rename(columns = {'pm25':'y'}, inplace = True)

datasets = [Delhi_o3, Delhi_no2, Delhi_so2, Delhi_pm10, Delhi_pm25]

In [6]:
pollutants = ['o3', 'no2', 'so2', 'pm10', 'pm25']

i=0

#EVALUATE MODEL FOR EVERY POLLUTANT
for data in datasets:
    test, forecast, error = fbprophet_predict(data, pollutants[i])
    
    i = i+1

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2020-10-11  0.180140    0.106330    0.253165
1 2020-10-18  0.175464    0.104994    0.246037
2 2020-10-25  0.168045    0.099351    0.244726
3 2020-11-01  0.160826    0.089633    0.231748
4 2020-11-08  0.155573    0.082987    0.229231
          ds         y
0 2020-10-11  0.234027
1 2020-10-18  0.149677
2 2020-10-25  0.146447
3 2020-11-01  0.143575
4 2020-11-08  0.091529
FB Prophet RMSE for Delhi[o3]: 0.06439994913932223

FB Prophet MSE for Delhi[o3]: 0.00414735344914729

FB Prophet MAE for Delhi[o3]: 0.05239568025710204



INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2020-10-11  0.116464    0.033652    0.200549
1 2020-10-18  0.131385    0.049335    0.217280
2 2020-10-25  0.142478    0.052090    0.226249
3 2020-11-01  0.142467    0.059637    0.228116
4 2020-11-08  0.135243    0.049707    0.213693
          ds         y
0 2020-10-11  0.182876
1 2020-10-18  0.266170
2 2020-10-25  0.364821
3 2020-11-01  0.374593
4 2020-11-08  0.419265
FB Prophet RMSE for Delhi[no2]: 0.16467175562223468

FB Prophet MSE for Delhi[no2]: 0.027116787099708977

FB Prophet MAE for Delhi[no2]: 0.14404407975864605



INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2020-10-11  0.213183    0.087190    0.343060
1 2020-10-18  0.223047    0.098381    0.352143
2 2020-10-25  0.230736    0.111350    0.359494
3 2020-11-01  0.232795    0.105811    0.365326
4 2020-11-08  0.230981    0.105817    0.362952
          ds         y
0 2020-10-11  0.239038
1 2020-10-18  0.249646
2 2020-10-25  0.272277
3 2020-11-01  0.231259
4 2020-11-08  0.277935
FB Prophet RMSE for Delhi[so2]: 0.06700278780181121

FB Prophet MSE for Delhi[so2]: 0.004489373573214541

FB Prophet MAE for Delhi[so2]: 0.05541519015682256



INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2020-10-11  0.090537    0.028851    0.155229
1 2020-10-18  0.096846    0.029549    0.161595
2 2020-10-25  0.111723    0.043907    0.171663
3 2020-11-01  0.126756    0.062361    0.194200
4 2020-11-08  0.130094    0.064196    0.198016
          ds         y
0 2020-10-11  0.119169
1 2020-10-18  0.145182
2 2020-10-25  0.173231
3 2020-11-01  0.229182
4 2020-11-08  0.329894
FB Prophet RMSE for Delhi[pm10]: 0.13343678740432285

FB Prophet MSE for Delhi[pm10]: 0.017805376232786455

FB Prophet MAE for Delhi[pm10]: 0.11690114237996845

          ds      yhat  yhat_lower  yhat_upper
0 2020-10-11  0.213100    0.119072    0.304146
1 2020-10-18  0.227977    0.134901    0.321942
2 2020-10-25  0.252608    0.154850    0.348390
3 2020-11-01  0.272917    0.184803    0.367335
4 2020-11-08  0.275299    0.173401    0.368798
          ds         y
0 2020-10-11  0.250718
1 2020-10-18  0.290415
2 2020-10-25  0.334291
3 2020-11-01  0.398537
4 2020-11-08  0.500653

## Monthly predictions

In [8]:
Delhi_Data_uni.index = Delhi_Data_uni['ds']
Delhi_Data_uni = Delhi_Data_uni.drop(['ds'],axis=1)

Delhi_Data_uni.index = pd.to_datetime(Delhi_Data_uni.index)
Delhi_monthly = Delhi_Data_uni.resample("1m").mean().bfill()

Delhi_monthly['ds'] = Delhi_monthly.index

In [9]:
Delhi_o3 = Delhi_monthly[['ds', 'o3']]
Delhi_o3.rename(columns = {'o3':'y'}, inplace = True)

Delhi_no2 = Delhi_monthly[['ds', 'no2']]
Delhi_no2.rename(columns = {'no2':'y'}, inplace = True)

Delhi_so2 = Delhi_monthly[['ds', 'so2']]
Delhi_so2.rename(columns = {'so2':'y'}, inplace = True)

Delhi_pm10 = Delhi_monthly[['ds', 'pm10']]
Delhi_pm10.rename(columns = {'pm10':'y'}, inplace = True)

Delhi_pm25 = Delhi_monthly[['ds', 'pm25']]
Delhi_pm25.rename(columns = {'pm25':'y'}, inplace = True)

datasets = [Delhi_o3, Delhi_no2, Delhi_so2, Delhi_pm10, Delhi_pm25]

In [10]:
pollutants = ['o3', 'no2', 'so2', 'pm10', 'pm25']

i=0

#EVALUATE MODEL FOR EVERY POLLUTANT
for data in datasets:
    test, forecast, error = fbprophet_predict(data, pollutants[i])
    
    i = i+1

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2020-10-31  0.134751    0.074575    0.197104
1 2020-11-30  0.140884    0.079035    0.207245
2 2020-12-31  0.132913    0.068817    0.198429
3 2021-01-31  0.172737    0.108687    0.233858
4 2021-02-28  0.178952    0.114175    0.243246
          ds         y
0 2020-10-31  0.178149
1 2020-11-30  0.141206
2 2020-12-31  0.138353
3 2021-01-31  0.125061
4 2021-02-28  0.127872
FB Prophet RMSE for Delhi[o3]: 0.03208520965740348

FB Prophet MSE for Delhi[o3]: 0.0010294606787595376

FB Prophet MAE for Delhi[o3]: 0.0264605617944996



INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2020-10-31  0.220035    0.138337    0.300212
1 2020-11-30  0.209227    0.120502    0.290121
2 2020-12-31  0.207870    0.130061    0.291103
3 2021-01-31  0.191962    0.112119    0.277635
4 2021-02-28  0.176685    0.092020    0.262753
          ds         y
0 2020-10-31  0.276768
1 2020-11-30  0.348371
2 2020-12-31  0.289429
3 2021-01-31  0.226752
4 2021-02-28  0.270242
FB Prophet RMSE for Delhi[no2]: 0.07935782332428408

FB Prophet MSE for Delhi[no2]: 0.0062976641227682865

FB Prophet MAE for Delhi[no2]: 0.06929623171378627



INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2020-10-31  0.258399    0.163544    0.351768
1 2020-11-30  0.237350    0.150228    0.325850
2 2020-12-31  0.212579    0.110155    0.297892
3 2021-01-31  0.224794    0.126548    0.321103
4 2021-02-28  0.242686    0.154362    0.333390
          ds         y
0 2020-10-31  0.239221
1 2020-11-30  0.253465
2 2020-12-31  0.200575
3 2021-01-31  0.169754
4 2021-02-28  0.230552
FB Prophet RMSE for Delhi[so2]: 0.07479067518971712

FB Prophet MSE for Delhi[so2]: 0.005593645095333767

FB Prophet MAE for Delhi[so2]: 0.059819654697993836



INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2020-10-31  0.216449    0.144014    0.292012
1 2020-11-30  0.206364    0.132372    0.279373
2 2020-12-31  0.204636    0.125717    0.278438
3 2021-01-31  0.120992    0.047997    0.195466
4 2021-02-28  0.108010    0.031047    0.191272
          ds         y
0 2020-10-31  0.156302
1 2020-11-30  0.236487
2 2020-12-31  0.183638
3 2021-01-31  0.163391
4 2021-02-28  0.148343
FB Prophet RMSE for Delhi[pm10]: 0.04972749393680863

FB Prophet MSE for Delhi[pm10]: 0.002472823653235339

FB Prophet MAE for Delhi[pm10]: 0.04048140669240041

          ds      yhat  yhat_lower  yhat_upper
0 2020-10-31  0.392451    0.281905    0.506359
1 2020-11-30  0.382726    0.263218    0.484403
2 2020-12-31  0.388888    0.280257    0.502835
3 2021-01-31  0.304044    0.193024    0.410321
4 2021-02-28  0.248992    0.139703    0.359090
          ds         y
0 2020-10-31  0.304889
1 2020-11-30  0.419257
2 2020-12-31  0.405201
3 2021-01-31  0.391874
4 2021-02-28  0.320972